# 2. Scraping
Scrape the [Top 10 Corporations by Year data](https://en.wikipedia.org/wiki/List_of_public_corporations_by_market_capitalization) from Wikipedia.

In [15]:
import bs4
import requests, re

In [16]:
url = 'http://en.wikipedia.org/wiki/List_of_public_corporations_by_market_capitalization'
res = requests.get(url)
soup = bs4.BeautifulSoup(res.text, 'html5lib')

In [17]:
tables = soup.find_all('table', {'class': 'wikitable'})
len(tables)

30

Filter out tables that aren't what we want. We'll check for the presence of a "rank" column and either "market value" or "first quarter".

In [18]:
def is_data_table(table):
    headers = table.find_all('th')
    header_text = ','.join([header.text for header in headers])
    if 'rank' not in header_text.lower():
        return False
    if 'market value' not in header_text.lower() and 'first quarter' not in header_text.lower():
        return False
    return True

tables = list(filter(is_data_table, tables))
len(tables)

28

Manual intervention...
For some reason, Wikipedia breaks out 2007 and 2006 into four separate tables, one for each quarter.

We're just going to manually remove these **except for Q1 2006**, which will have to serve as our record for 2005.
Because 2005 is completely missing. Of course.

In [19]:
tables = [table for index, table in enumerate(tables)
          if index not in (13, 14, 15, 17, 18)]

### Get the company names out of the table cells, 2008-2018

This function will return the first line of text from any cell with a newline character.
This handles the formatting for the tables from 2008 to 2018.

In [20]:
def extract_text_from_td(td):
    text = td.text
    regex = re.compile('(.*)\n.*')
    match = regex.search(text)
    if match is not None:
        return match.group(1)

extract_text_from_td(tables[0].find_all('td')[2])

'Apple Inc.'

Run the function on all table cells. Store the company names in one *set* per year, so that removed duplicates.

In [21]:
company_names = {}
# Run only on the first 11 tables (the ones formatted this way)
for index, table in enumerate(tables[:11]):
    tds = table.find_all('td')
    text = map(extract_text_from_td, tds)
    # Keep only results where company names were found.
    text = filter(lambda x: x is not None and len(x) > 0, text)
    company_names[index] = set(text)

company_names[0]

{'Alibaba Group',
 'Alphabet Inc.',
 'Amazon.com',
 'Apple Inc.',
 'Berkshire Hathaway',
 'Facebook',
 'JPMorgan Chase',
 'Johnson & Johnson',
 'Microsoft',
 'Tencent'}

### Get the company names out of the table cells, 1996-2007
This function gets the text out of the second table cell in every non-header row.
From 1996 to 2007, this is where to find the company name.

In [22]:
def extract_text_from_tr(tr):
    # The company name is always in the 2nd column.
    tds = tr.find_all('td')
    if len(tds) > 0:
        td = tds[1]
        return td.text

Run it on every row.

In [23]:
for index, table in enumerate(tables[11:]):
    # This time, break it down by rows not columns.
    trs = table.find_all('tr')
    text = map(extract_text_from_tr, trs)
    # Keep only results where company names were found.
    text = filter(lambda x: x is not None and len(x) > 0, text)
    # Add 11 to the index because we've already inserted the first 11 years above.
    company_names[index + 11] = set(text)

company_names[0]

{'Alibaba Group',
 'Alphabet Inc.',
 'Amazon.com',
 'Apple Inc.',
 'Berkshire Hathaway',
 'Facebook',
 'JPMorgan Chase',
 'Johnson & Johnson',
 'Microsoft',
 'Tencent'}

We should have as many keys in the dictionary as we had tables to start with.

In [24]:
assert len(tables) == len(company_names.keys())

Translate this dictionary so the keys are the relevant years, so 1996-2018 (not 0-27).
Note that our tables are currently listed in descending order, where index 0 == year 2018.
While we're at it, convert the company names into lists instead of sets - simply because I find lists easier to work with.

In [25]:
companies_by_year = {2018-key: list(company_names[key])
                     for key in company_names.keys()}
companies_by_year[2006]

['AT&T',
 'Citigroup',
 'Microsoft',
 'General Electric',
 'Exxon Mobil',
 'China Mobile',
 'Industrial and Commercial Bank of China',
 'Bank of America',
 'Royal Dutch Shell',
 'Gazprom']

How many companies do we have per year?

In [27]:
for year in sorted(companies_by_year.keys()):
    n_companies = len(companies_by_year[year])
    print(f'{year}: {n_companies}')

1996: 5
1997: 10
1998: 10
1999: 10
2000: 10
2001: 10
2002: 10
2003: 10
2004: 10
2005: 10
2006: 10
2007: 10
2008: 15
2009: 15
2010: 15
2011: 14
2012: 13
2013: 14
2014: 15
2015: 16
2016: 13
2017: 12
2018: 10


Store this scraped data as JSON. We'll use it in the next notebook.

In [38]:
import json, os
current_dir = os.path.join(os.environ['HOME'],
                           'data_science/syntact/buffett_emergence/')
json_path = os.path.join(current_dir, 'companies_by_year.json')
with open(json_path, 'w') as f:
    json.dump(companies_by_year, f)